In [26]:
import json
import os
import numpy as np
import pandas as pd

emotion_label_policy = {'angry': 0, 'anger': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3, 'happines': 3, 'happiness': 3, 
    'sad': 4, 'sadness': 4,
    'surprise': 5, 'surprised': 5, 
    'neutral': 6,
    'excited': 7,
    'frustrated': 8,}

emotion_label_num_to_emotion = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'excited', 'frustrated']

In [27]:
# json 데이터셋 파일 열기
datasets = ['../dataset/dailydialog_train.json', '../dataset/dailydialog_valid.json', '../dataset/dailydialog_test.json']

# 각 데이터셋의 데이터를 하나의 리스트로 합치기
with open(datasets[0], 'r', encoding='utf-8') as f:
    train = json.load(f)
with open(datasets[1], 'r', encoding='utf-8') as f:
    valid = json.load(f)
with open(datasets[2], 'r', encoding='utf-8') as f:
    test = json.load(f)

# concate 3 dictionaries 
valid.update(test)
train.update(valid)
total_data = train

# dataset 폴더에 'dailydialog_total.json' 파일이 없으면, total_data를 json 파일로 저장
if not os.path.isfile('../dataset/dailydialog_total.json'):
    with open('../dataset/dailydialog_total.json', 'w', encoding='utf-8') as f:
        json.dump(total_data, f, ensure_ascii=False, indent='\t')
        
dialog_ids = list(total_data.keys())

## Dialog_table_dict 만드는 코드

In [148]:
# table1_columns = ['dialog_id', 'emotion', 'num_utterances']
table_columns = ['dialog_id', 'turn', 'speaker', 'utterance', 'emotion', 'cause turn', 'cause span', 'cause_type', 'explanation']

# 각 dialog가 들어갈 table 선언하고 내용물 채우기 (column은 table_columns)
# key는 dialog_id와 uid의 조합
# value는 speaker, utterance, emotion, cause turn, cause span
# table: dialog_id, uid, speaker, utterance, emotion, cause turn, cause span

# 각 row를 저장할 데이터프레임
dialog_table_all = pd.DataFrame(columns=table_columns)
for dialog_id in total_data.keys():
    dialog = total_data[dialog_id][0]
    for utterance in dialog:
        turn = utterance['turn']
        uid = f'{dialog_id}_{turn}'
        
        utterance['emotion'] = emotion_label_num_to_emotion[emotion_label_policy[utterance['emotion']]]
        if utterance['emotion'] != 'neutral':
            if 'explanation' in utterance.keys():
                row = pd.DataFrame([[dialog_id, turn, utterance['speaker'], utterance['utterance'], utterance['emotion'], \
                    utterance['expanded emotion cause evidence'], utterance['expanded emotion cause span'], utterance['type'], utterance['explanation']]], columns=table_columns)
            else:
                row = pd.DataFrame([[dialog_id, turn, utterance['speaker'], utterance['utterance'], utterance['emotion'], \
                    utterance['expanded emotion cause evidence'], utterance['expanded emotion cause span'], utterance['type'], '']], columns=table_columns)
                
        else:
            row = pd.DataFrame([[dialog_id, turn, utterance['speaker'], utterance['utterance'], utterance['emotion'], '', '', '', '']], columns=table_columns)
        dialog_table_all = pd.concat([dialog_table_all, row], ignore_index=True)
        
        
    # table1: dialog_id, emotion, num_utterances

dialog_table_dict = {}
for dialog_id in total_data.keys():
    dialog_table_dict[dialog_id] = dialog_table_all[dialog_table_all['dialog_id'] == dialog_id]


## Dialog_table 만드는 코드

In [171]:
# dialog의 각 감정, dialog의 길이에 따라서 dialog 단위로 attribute를 설정
dialog_table_dict.keys()

table_columns = ['dialog_id', 'emotion', 'len']
dialog_table = pd.DataFrame(columns=table_columns)

for dialog_id in dialog_table_dict.keys():
    dialog = dialog_table_dict[dialog_id]
    # emotion은 dialog['emotion']의 최빈값 중 neutral이 아닌 값으로 정함
    emotion = 'neutral'
    for emotion_appearance in dialog['emotion'].value_counts().index:
        if emotion_appearance == 'neutral':
            continue
        if emotion_appearance != 'neutral':
            emotion = emotion_appearance#dialog['emotion'].value_counts().index[0]
            break
    # emotion = dialog['emotion'].value_counts().index[0]
    len_ = len(dialog)
    row = pd.DataFrame([[dialog_id, emotion, len_]], columns=table_columns)
    dialog_table = pd.concat([dialog_table, row], ignore_index=True)

In [172]:
# dialog_table_dict를 펴서 하나의 DataFrame으로 만들기
table_columns = ['dialog_id', 'turn', 'speaker', 'utterance', 'emotion', 'cause turn', 'cause span', 'cause_type', 'explanation']
dialog_table_all = pd.DataFrame(columns=table_columns)
for dialog_id in dialog_table_dict.keys():
    dialog_table_all = pd.concat([dialog_table_all, dialog_table_dict[dialog_id]], ignore_index=True)

## 정리한 데이터
<b><i>dialog_table_dict</i></b>: dialog_id를 key로 갖는 dictionary. 각 dialog는 DataFrame 형태로 utterance들을 가짐  <br>
<b><i>dialog_table</i></b>: DataFrame 타입. Dialog 단위로 (dialog_id, emotion(neutral제외), 대화길이(발화수))를 가짐 <br>

In [28]:
dialog_table_dict['tr_4466']

NameError: name 'dialog_table_dict' is not defined

In [174]:
# table['emotion']의 분포를 보기
dialog_table['emotion'].value_counts()

happy       789
angry       115
sad          83
surprise     69
disgust      34
fear         16
Name: emotion, dtype: int64

In [175]:
type(dialog_table_all)

pandas.core.frame.DataFrame

In [188]:
# 정리한 데이터를 csv로 저장
# 공백 줄 없이

with open('../dataset/dailydialog_table_all_before_processed.csv', 'w', encoding='utf-8') as f:
    dialog_table_all.to_csv(f, index=False)
with open('../dataset/dailydialog_table_before_processed.csv', 'w', encoding='utf-8') as f:
    dialog_table.to_csv(f, index=False)

In [25]:
dialog_table

NameError: name 'dialog_table' is not defined

In [190]:
path_list = ['../dataset/dailydialog_table_before_processed.csv', '../dataset/dailydialog_table_all_before_processed.csv']
output_path = ['../dataset/dailydialog_table.csv', '../dataset/dailydialog_table_all.csv']

# 텍스트 파일을 불러와서 짝수 번째 줄을 제거하고 다시 저장하는 코드
for path, output_path in zip(path_list, output_path):
    all_lines = ''
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # lines의 짝수 번째를 제거
    for i in range(len(lines)):
        if i % 2 == 0:
            all_lines += lines[i]
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(all_lines)

In [22]:
# csv 파일 불러와서, text의 특수문자 앞의 공백을 지우기
# 특수문자 앞에 공백이 있으면, 토큰화할 때 특수문자가 따로 분리되는 문제가 있음
# 특수문자 앞의 공백을 지우고 다시 저장
path_list = ['../dataset/dailydialog_table_all.csv']
output_path = ['../dataset/dailydialog_table_all_processed.csv']

import re

# csv파일 불러오기
for path, output_path in zip(path_list, output_path):
    # dataFrame으로 불러옴
    df = pd.read_csv(path)
    
    for j in range(len(df)):
        # 특수문자 앞의 공백을 지움
        df['utterance'][j] = re.sub(r'\s+([.,!?])', r'\1', df['utterance'][j])
        
        # cause span도 똑같이 list 속 각 문자열의 특수문자 앞 공백 지움
        cause_span = df['cause span'][j]
        if type(cause_span) == str:
            cause_span = cause_span.split(',')
            for i in range(len(cause_span)):
                cause_span[i] = re.sub(r'\s+([.,!?])', r'\1', cause_span[i])
            df['cause span'][j] = ','.join(cause_span)
        

C:\Users\Admin\AppData\Local\Temp\ipykernel_11028\2479855666.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['utterance'][j] = re.sub(r'\s+([.,!?])', r'\1', df['utterance'][j])
C:\Users\Admin\AppData\Local\Temp\ipykernel_11028\2479855666.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cause span'][j] = ','.join(cause_span)


In [24]:
df.to_csv(output_path, index=False)

In [23]:
df

,dialog_id,turn,speaker,utterance,emotion,cause turn,cause span,cause_type,explanation
0,tr_4466,1,A,"Hey, you wanna see a movie tomorrow?",happy,[1],['see a movie tomorrow?'],['no-context'],NaN
1,tr_4466,2,B,Sounds like a good plan. What do you want to see?,happy,[1],['see a movie tomorrow?'],['inter-personal'],NaN
2,tr_4466,3,A,How about Legally Blonde.,neutral,NaN,NaN,NaN,NaN
3,tr_4466,4,B,"Ah, my girlfriend wanted to see that movie. I ...",neutral,NaN,NaN,NaN,NaN
4,tr_4466,5,A,Isn't that a scary movie?,neutral,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
11099,tr_10060,6,B,"Yes, of course.",happy,[5],['Can you give her a message'],['inter-personal'],NaN
11100,tr_10060,7,A,Please tell her to give Colin a call when she ...,neutral,NaN,NaN,NaN,NaN
11101,tr_10060,8,B,Sure.,happy,"[5, 7]","['Can you give her a message', 'Please tell he...",['inter-personal'],NaN
11102,tr_10060,9,A,Thank you for your help.,happy,"[5, 7, 8]","['Can you give her a message', 'Please tell he...",['hybrid'],NaN
